<a href="https://colab.research.google.com/github/ANSHAY/deeplearning/blob/master/17_chatterbox/main_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chatty the chatterbox

## Import Libraries

In [0]:
import tensorflow as tf
import matplotlib.pyplot as plt
import random
import numpy as np

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

## Define Parameters

In [0]:
dataset_type = 'conversation'

files = [dataset_type + '_train.txt']
save_file_path = dataset_type + '_predicted.txt'
PADDING = 'pre'
TRUNC = 'pre'
MAXLEN = 7

VOCAB_SIZE = 10000
EMB_DIM = 128

OPTIMIZER = 'adam'
LOSS = 'categorical_crossentropy'
METRICS = ['acc']
EPOCHS = 1000
BATCH_SIZE = 128
VAL_SPLIT = 0.1

SEEDER = 'Hi, Whats up'    ## Seeder word to start prediction of poetry

model_name = dataset_type + "_model_colab.h5"

data = "This is some random statement \n being used as placeholder for the actual data that is to be \n imported later from a file."

## Load Data

In [0]:
corpus = ''
for file in files:
    with open(file) as f:
        data = f.read()
        corpus += data + '\n'
        f.close()
#data = data.replace('\n', ' \n<>')
sentences = corpus.lower().split('\n')
print(len(sentences))
print(sentences[0])
print(len(sentences[0]))

FileNotFoundError: ignored

## Tokenize Data

In [0]:
tokenizer = Tokenizer()          #filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t')
tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index
total_words = len(word_index) + 1
print(total_words)
print(word_index['a'])

2256
3


In [0]:
## create reverse_word_index
reverse_word_index = {}
for word, i in word_index.items():
    reverse_word_index[i] = word
print(reverse_word_index[1])

you


## Change Sentences to Sequences

In [0]:
sequences = tokenizer.texts_to_sequences(sentences)
padded_seq = pad_sequences(sequences, padding=PADDING, truncating=TRUNC, maxlen=MAXLEN)
print(padded_seq[0])

[0 0 0 0 0 0 0]


## Extract trainX and trainY from sequences

In [0]:
trainX = padded_seq[:-1,:]
trainY = padded_seq[1:,:]

INP_LEN = trainX.shape[1]
OUT_LEN = trainY.shape[1]
trainY = tf.keras.utils.to_categorical(trainY, num_classes=total_words)

print(trainX.shape)
print(trainY.shape)

(2309, 7)
(2309, 7, 2256)


In [0]:
print(padded_seq[1])
print(trainY[0])

[  0   0   0   0   7   6 418]
[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


## Define Callback

In [0]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epochs, log={}):
        if (log.get('acc')>0.90):
            self.model.stop_training = True
            print("\n Stopped training since model reached accuracy of 90%")
callback = myCallback()

## Define Model

In [0]:
LOAD_MODEL = False

if LOAD_MODEL == True:
    try:
        model = tf.keras.models.load_model(model_name)
        print("\nModel loaded successfully")
    except:
        print("\nModel not loaded. Building new...")
        LOAD_MODEL = False
if LOAD_MODEL == False:
    model = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(total_words, EMB_DIM, input_length=INP_LEN),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True)),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
        tf.keras.layers.Dense(total_words, activation='softmax')
    ])
    


Model not loaded. Building new...
Instructions for updating:
Colocations handled automatically by placer.


In [0]:
model.compile(optimizer=OPTIMIZER, loss=LOSS, metrics=METRICS)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 7, 128)            288768    
_________________________________________________________________
bidirectional (Bidirectional (None, 7, 256)            263168    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 7, 128)            164352    
_________________________________________________________________
dense (Dense)                (None, 7, 2256)           291024    
Total params: 1,007,312
Trainable params: 1,007,312
Non-trainable params: 0
_________________________________________________________________


## Train Model

In [0]:
history = model.fit(trainX, trainY, validation_split=VAL_SPLIT, verbose=1, epochs = EPOCHS, shuffle=False, batch_size=BATCH_SIZE, callbacks=[callback])

Train on 2078 samples, validate on 231 samples
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Epoch 1/1000
2078/2078 [==============================] - 10s 5ms/sample - loss: 7.6453 - acc: 0.2573 - val_loss: 7.2569 - val_acc: 0.1979
Epoch 2/1000
2078/2078 [==============================] - 4s 2ms/sample - loss: 5.8147 - acc: 0.2784 - val_loss: 5.6399 - val_acc: 0.1979
Epoch 3/1000
2078/2078 [==============================] - 4s 2ms/sample - loss: 4.8447 - acc: 0.2784 - val_loss: 5.5874 - val_acc: 0.1979
Epoch 4/1000
2078/2078 [==============================] - 4s 2ms/sample - loss: 4.7432 - acc: 0.2784 - val_loss: 5.5843 - val_acc: 0.1979
Epoch 5/1000
2078/2078 [==============================] - 4s 2ms/sample - loss: 4.6983 - acc: 0.2787 - val_loss: 5.6058 - val_acc: 0.2010
Epoch 6/1000
2078/2078 [==============================] - 4s 2ms/sample - loss: 4.6626 - acc: 0.2817 - val_loss: 5.6251 - val_acc: 0.200

Epoch 56/1000
2078/2078 [==============================] - 4s 2ms/sample - loss: 3.9656 - acc: 0.3050 - val_loss: 6.3973 - val_acc: 0.1763
Epoch 57/1000
2078/2078 [==============================] - 4s 2ms/sample - loss: 3.9586 - acc: 0.3070 - val_loss: 6.3672 - val_acc: 0.1769
Epoch 58/1000
2078/2078 [==============================] - 4s 2ms/sample - loss: 3.9417 - acc: 0.3064 - val_loss: 6.3168 - val_acc: 0.1633
Epoch 59/1000
2078/2078 [==============================] - 4s 2ms/sample - loss: 3.9351 - acc: 0.3063 - val_loss: 6.3277 - val_acc: 0.1626
Epoch 60/1000
2078/2078 [==============================] - 4s 2ms/sample - loss: 3.9452 - acc: 0.3052 - val_loss: 6.3989 - val_acc: 0.1744
Epoch 61/1000
2078/2078 [==============================] - 4s 2ms/sample - loss: 3.9488 - acc: 0.3090 - val_loss: 6.4110 - val_acc: 0.1793
Epoch 62/1000
2078/2078 [==============================] - 4s 2ms/sample - loss: 3.9393 - acc: 0.3032 - val_loss: 6.3480 - val_acc: 0.1645
Epoch 63/1000
2078/2078 [==

Epoch 115/1000
2078/2078 [==============================] - 4s 2ms/sample - loss: 3.4130 - acc: 0.3496 - val_loss: 6.6615 - val_acc: 0.1583
Epoch 116/1000
2078/2078 [==============================] - 4s 2ms/sample - loss: 3.3961 - acc: 0.3536 - val_loss: 6.7252 - val_acc: 0.1657
Epoch 117/1000
2078/2078 [==============================] - 4s 2ms/sample - loss: 3.3913 - acc: 0.3560 - val_loss: 6.7315 - val_acc: 0.1645
Epoch 118/1000
2078/2078 [==============================] - 4s 2ms/sample - loss: 3.3791 - acc: 0.3529 - val_loss: 6.6520 - val_acc: 0.1515
Epoch 119/1000
2078/2078 [==============================] - 4s 2ms/sample - loss: 3.3866 - acc: 0.3541 - val_loss: 6.6401 - val_acc: 0.1490
Epoch 120/1000
2078/2078 [==============================] - 4s 2ms/sample - loss: 3.3765 - acc: 0.3539 - val_loss: 6.7469 - val_acc: 0.1639
Epoch 121/1000
2078/2078 [==============================] - 4s 2ms/sample - loss: 3.3788 - acc: 0.3570 - val_loss: 6.8007 - val_acc: 0.1707
Epoch 122/1000
2078/

2078/2078 [==============================] - 4s 2ms/sample - loss: 2.7696 - acc: 0.4247 - val_loss: 7.0701 - val_acc: 0.1589
Epoch 174/1000
2078/2078 [==============================] - 4s 2ms/sample - loss: 2.7592 - acc: 0.4265 - val_loss: 7.0191 - val_acc: 0.1515
Epoch 175/1000
2078/2078 [==============================] - 4s 2ms/sample - loss: 2.7645 - acc: 0.4246 - val_loss: 7.0407 - val_acc: 0.1490
Epoch 176/1000
2078/2078 [==============================] - 4s 2ms/sample - loss: 2.7562 - acc: 0.4264 - val_loss: 7.1270 - val_acc: 0.1558
Epoch 177/1000
2078/2078 [==============================] - 4s 2ms/sample - loss: 2.7535 - acc: 0.4280 - val_loss: 7.1441 - val_acc: 0.1614
Epoch 178/1000
2078/2078 [==============================] - 4s 2ms/sample - loss: 2.7689 - acc: 0.4269 - val_loss: 7.0975 - val_acc: 0.1639
Epoch 179/1000
2078/2078 [==============================] - 4s 2ms/sample - loss: 2.7704 - acc: 0.4286 - val_loss: 7.0397 - val_acc: 0.1478
Epoch 180/1000
2078/2078 [=========

2078/2078 [==============================] - 4s 2ms/sample - loss: 2.2376 - acc: 0.5219 - val_loss: 7.5450 - val_acc: 0.1596
Epoch 232/1000
2078/2078 [==============================] - 4s 2ms/sample - loss: 2.2129 - acc: 0.5230 - val_loss: 7.4265 - val_acc: 0.1404
Epoch 233/1000
2078/2078 [==============================] - 4s 2ms/sample - loss: 2.1705 - acc: 0.5315 - val_loss: 7.3999 - val_acc: 0.1453
Epoch 234/1000
2078/2078 [==============================] - 4s 2ms/sample - loss: 2.1460 - acc: 0.5361 - val_loss: 7.4973 - val_acc: 0.1503
Epoch 235/1000
2078/2078 [==============================] - 4s 2ms/sample - loss: 2.1147 - acc: 0.5431 - val_loss: 7.5141 - val_acc: 0.1552
Epoch 236/1000
2078/2078 [==============================] - 4s 2ms/sample - loss: 2.0947 - acc: 0.5449 - val_loss: 7.4545 - val_acc: 0.1472
Epoch 237/1000
2078/2078 [==============================] - 4s 2ms/sample - loss: 2.0699 - acc: 0.5509 - val_loss: 7.4554 - val_acc: 0.1447
Epoch 238/1000
2078/2078 [=========

2078/2078 [==============================] - 4s 2ms/sample - loss: 1.5649 - acc: 0.6511 - val_loss: 7.8714 - val_acc: 0.1367
Epoch 290/1000
2078/2078 [==============================] - 4s 2ms/sample - loss: 1.5565 - acc: 0.6550 - val_loss: 7.9157 - val_acc: 0.1391
Epoch 291/1000
2078/2078 [==============================] - 4s 2ms/sample - loss: 1.5386 - acc: 0.6601 - val_loss: 7.9933 - val_acc: 0.1429
Epoch 292/1000
2078/2078 [==============================] - 5s 2ms/sample - loss: 1.5288 - acc: 0.6603 - val_loss: 7.9535 - val_acc: 0.1404
Epoch 293/1000
2078/2078 [==============================] - 5s 2ms/sample - loss: 1.5183 - acc: 0.6642 - val_loss: 7.9123 - val_acc: 0.1367
Epoch 294/1000
2078/2078 [==============================] - 5s 2ms/sample - loss: 1.5151 - acc: 0.6622 - val_loss: 7.9586 - val_acc: 0.1385
Epoch 295/1000
2078/2078 [==============================] - 5s 2ms/sample - loss: 1.5098 - acc: 0.6649 - val_loss: 8.0324 - val_acc: 0.1435
Epoch 296/1000
2078/2078 [=========

KeyboardInterrupt: 

## Save Model

In [0]:
model.save(model_name)

## Plot Results

In [0]:
acc = history.history['acc']
val_acc = history.history['val_acc']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(EPOCHS)

plt.plot(epochs, acc, 'r')
plt.plot(epochs, val_acc, 'b')
plt.xlabel('EPOCHS')
plt.ylabel('Accuracies')
plt.legend('Train Acc', 'Val Acc')
plt.figure()

plt.plot(epochs, loss, 'r')
plt.plot(epochs, val_loss, 'b')
plt.xlabel('EPOCHS')
plt.ylabel('Losses')
plt.legend('Train Loss', 'Val Loss')
plt.figure()

NameError: name 'history' is not defined

## Generate Poetry by iterating over the predictor from the model

In [0]:
reverse_word_index[0] = ' '

In [0]:
sentence = SEEDER.lower()
corpus = SEEDER.lower()
new_line = False
print (sentence, end='\n')

sentence = input()
while(sentence != 'stop'):
    corpus += '\n' + sentence
    seq = tokenizer.texts_to_sequences([sentence])
    seq = pad_sequences(seq, padding=PADDING, truncating=TRUNC, maxlen=INP_LEN)
    pred_seq = model.predict(seq)
    pred_seq = np.argmax(pred_seq, axis=2)
    pred_sent = [reverse_word_index[x] for x in pred_seq[0]]
    pred_sent = ' '.join(pred_sent)
    print(pred_sent)
    corpus += '\n' + pred_sent
    sentence = input()
print ("Bye, See ya later!!!")


hi, whats up
how are you
        you you you
i i i 
  you you you to to weekend
lets plaay on weekend
      you you you you
what the hell
        you you you
dont say you
    i you you to you
lol
        you you you
stop
Bye, See ya later!!!


## Save Poetry in a file

In [0]:
try:
    with open(save_file_path, 'w') as f:
        f.write(corpus)
        f.close()
except:
    print("\nWrite was Unsuccessful")